In [1]:
from rich import print
import json
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
import os
from openai import OpenAI
import pandas as pd
from tqdm import tqdm

In [2]:
_ = load_dotenv(find_dotenv())
api_key = os.environ['OPENAI_API_KEY']

base_url = 'https://integrate.api.nvidia.com/v1'
client = OpenAI(base_url=base_url)

In [3]:
pwd

'/notebooks/ABSA/notebooks/sentiment-notebooks'

In [4]:
example1 = "المحتوى أكثر من رائع"

In [5]:
system_message = """
You will receive a review about educational services, and your task is to classify its sentiment as positive, neutral, or negative.
The review may be in English language, in Arabic language ,or both.
The only allowed output options are [positive, neutral, negative]
"""

In [6]:
user_template = """
In context of sentiment analysis, you have to classify the sentiment of the following review into positive, neutral, or negative.

review:
###
{}
###

output: 
"""

In [7]:
def format_user_message(template, text):
    return template.format(text)


In [8]:
def format_one_example(review):
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": format_user_message(user_template, review)
            }
        ]
    }

In [9]:
print(format_one_example(example1))

{
    'messages': [
        {
            'role': 'system',
            'content': '\nYou will receive a review about educational services, and your task is to classify its 
sentiment as positive, neutral, or negative.\nThe review may be in English language, in Arabic language ,or 
both.\nThe only allowed output options are [positive, neutral, negative]\n'
        },
        {
            'role': 'user',
            'content': '\nIn context of sentiment analysis, you have to classify the sentiment of the following 
review into positive, neutral, or negative.\n\nreview:\n###\nالمحتوى أكثر من رائع\n###\n\noutput: \n'
        }
    ]
}

In [10]:
model_name = 'meta/llama3-70b-instruct'
# model_name = 'meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo'
def get_completion(review, model_name):
    completion = client.chat.completions.create(
        model=model_name,
        temperature=0,
        messages=format_one_example(review)['messages']
    )
    
    return completion.choices[0].message.content

In [11]:
def predict_gpt(item, model_name=model_name):
    label = get_completion(item, model_name)
    # result = json.loads(completion)
    # pred_labels = [result.get(token, 'O') for token in item['tokens']]
    # pred_labels = [v for k, v in result.items()]
    # organizations = result['Organizations']
    return label

In [12]:
example1 = "المحتوى أكثر من رائع"

In [13]:
# predict_gpt(example1) # positive

In [14]:
train_data_path = '../../data/result/train_df.csv'
test_data_path = '../../data/result/test_df.csv'
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [15]:
train_data.head()

,column,text,topic_id,topic_name
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول


In [16]:
tqdm.pandas()

print('Annotate train data...')
# train_data['sentiment'] = train_data['text'].progress_apply(predict_gpt)
print('Annotate test data...')
test_data['sentiment'] = test_data['text'].progress_apply(predict_gpt)

Annotate train data...

Annotate test data...

100%|██████████| 634/634 [01:46<00:00,  5.95it/s]


In [17]:
# train_data['sentiment'] = train_data['sentiment'].progress_apply(str.capitalize)
test_data['sentiment'] = test_data['sentiment'].progress_apply(str.capitalize)

100%|██████████| 634/634 [00:00<00:00, 286118.87it/s]


In [18]:
test_data.head()

,column,text,topic_id,topic_name,sentiment
0,like,أكثر ما أعجبني: المقرر ساعدني جدا في فهم نظام ...,1,دكتور المقرر,Positive
1,dislike,أكثر ما لم يعجبني: المقرر يحتوي على مواضيع ساب...,6,طريقة تدريس و تقديم المادة,Negative
2,dislike,أكثر ما لم يعجبني: أسلوب شرح المقرر,5,المتطلبات و المهام و الدرجات,Negative
3,add_suggestions,اقتراحاتي للإضافة: everything is spectacular a...,2,None,Positive
4,like,أكثر ما أعجبني: معلمة المادة متمكنة وتستجيب عل...,1,دكتور المقرر,Positive


In [19]:
test_data['sentiment'].value_counts() / len(test_data)

Positive    0.553628
Negative    0.268139
Neutral     0.178233
Name: sentiment, dtype: float64

In [ ]:
# test_data['sentiment'].value_counts() / len(test_data)

In [20]:
# train_data.to_csv('../../data/result/train_with_sentiment_df_llm_llama_3.csv', index=False)
test_data.to_csv('../../data/result/test_with_sentiment_df_llm_llama_3.csv', index=False)